In [1]:
# !pip install pypdf
# !pip install langchain
# !pip install google-cloud-storage
# !pip install openai
# !pip install tiktoken
# !pip install psycopg2
# !pip install cloud-sql-python-connector
# !pip install asyncio asyncpg
# !pip install pgvector

In [2]:
import os
import openai
import sys
from datetime import datetime
from google.cloud.sql.connector import Connector
import asyncio
import asyncpg
import pickle
from pgvector.asyncpg import register_vector


from google.cloud import storage
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
query = """
CREATE TABLE IF NOT EXISTS Document_test (
    document_id SERIAL PRIMARY KEY,
    tag VARCHAR(255),
    content TEXT,
    embedding VECTOR(1536)
);
"""

async def connect_execute(query):
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )

    await conn.execute(query)
    print("Query created successfully.")

    await conn.close()

loop = asyncio.get_event_loop()

task = loop.create_task(connect_execute(query))

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from google.cloud import storage

storage_client = storage.Client.from_service_account_json(str(os.getenv('STORAGE_SERVICE_JSON')))

bucket_name = 'bort_storage'

pdf_file_name = 'Orthian Services.pdf'

bucket = storage_client.get_bucket(bucket_name)

blob = bucket.blob(pdf_file_name)

blob.download_to_filename(pdf_file_name.split('/')[-1])

print(f"The file {pdf_file_name} has been downloaded.")


loaders = [
    PyPDFLoader(pdf_file_name.split('/')[-1]),
    # next
]

docs = []

for loader in loaders:
    docs.extend(loader.load())

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size = 1536,
    chunk_overlap = 0
)
texts = text_splitter.split_documents(docs)

The file Orthian Services.pdf has been downloaded.


In [5]:
texts[0].page_content

'"We\npride\nourselves\nto\nbecome\na\ntrusted\ndigital\npartner\nand\noffer\nexceptional\nservices\nto\norganizations\nat\nall\nsizes\nincluding:\n1.\nBusiness\nInnovation\n+Al;\n2.\nWebsite\nDesign\n&\nDevelopment;\n3.\nSoftware\nDevelopment;\n4.\nIT\nServices;\n5.\nMarketing\nDesign"\n"With\nOrthian,\nyou\ngain\nmore\nthan\njust\na\nproduct\nstudio.\nOrthian\nis\nyour\ntrusted\nally\nin\ndriving\nbusiness\nsuccess\nthrough\ndigital\ninnovation.\nWe\nmake\ninnovation\nwith\ntangible\nimpact:\nInnovation\nis\nmore\nthan\njust\na\nbuzzword\nto\nus;\nit\'s\nabout\ndelivering\nreal-world\nresults.\nFrom\nboosting\noperational\nefficiency\nwith\nsoftware\ndevelopment\nto\ndriving\ncustomer\nengagement\nthrough\ncaptivating\ndesign,\nwe\'re\ncommitted\nto\ncreating\ninnovation\nthat\ntangibly\nenhances\nyour\nbusiness\'s\nbottom\nline.\nWe\ncustomize\nsolutions\nfor\nyour\nscale:\nWe\nunderstand\nthat\nbusinesses\nvary\nin\nsize\nand\nneeds.\nWhether\nyou\nare\na\nstartup\naiming\nfor\nrap

In [6]:
insert_query = """
    INSERT INTO Document_test (tag, content, embedding) VALUES ($1, $2, $3);
"""

async def connect_execute(insert_query, splits):
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )

    
    await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
    await register_vector(conn)

    embeddings = OpenAIEmbeddings()
    doc_vector = embeddings.embed_documents([t.page_content for t in texts])

    i = -1 
    for split in splits:
        try:
            i = i + 1
            await conn.execute(insert_query, 'Company info', split.page_content, doc_vector[i])
            print("added successfully.")
        except Exception as e:
            print(f"Failed to add split: {e}")

    await conn.close()

loop = asyncio.get_event_loop()

task = loop.create_task(connect_execute(insert_query, texts))

In [7]:
import pandas as pd

query = """
SELECT * FROM Document_test limit 1;
"""

async def connect_execute(query):
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )

    rows = await conn.fetch(query)
    print("Query executed successfully.")

    column_names = ['document_id', 'tag', 'content', 'embedding']

    df = pd.DataFrame(rows, columns=column_names)

    await conn.close()

    return df

loop = asyncio.get_event_loop()

task = loop.create_task(connect_execute(query))

df = await task

print(df)

Query created successfully.
added successfully.
added successfully.
added successfully.
Query executed successfully.
   document_id           tag  \
0            1  Company info   

                                             content  \
0  "We\npride\nourselves\nto\nbecome\na\ntrusted\...   

                                           embedding  
0  [0.0028839186,-0.011037433,-0.014199882,-0.014...  
